<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

# Microestructuras y Sistemas de Traiding 

## Autores: Ariadna Galindo y Bruno Pimentel 
### Otoño 2021

# 0.Librerias y dependencias 

## 0.1 Librerias necesarias

La lista completa requirements se encuentra en *requirements.txt* 

* certifi==2021.5.30
* charset-normalizer==2.0.4
* idna==3.2
* lxml==4.6.3
* mpmath==1.2.1
* numpy==1.21.2
* pandas==1.3.2
* pandas-datareader==0.10.0
* python-dateutil==2.8.2
* pytz==2021.1
* requests==2.26.0
* six==1.16.0
* sympy==1.8
* urllib3==1.26.6

In [1]:
#importamos librerias 
import math as m
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import pandas as pd
import os 
import sys 
import glob

## 0.2 Dependencias 
Para correr correctamente este notebook se requiere tener la carpeta /files con los archivos de NAFTRAC tal y como la incluimos en el repositorio. Adicionalmente se recomienda correr primero los siguientes archivos en el siguiente orden: functions.py>data.py>main.py

# 1. Introducción 

Para este primer Laboratorio se tiene un millón de pesos para invertir. Se deben usar dos estrategias diferentes; activa y pasiva. En la estrategia pasiva se emplea el 100% del capital en títulos del NAFTRAC, ya que este en un ETF del mercado mexicano que replica el movimiento del índice del S&P/BMV IPC. Mientras que en la estrategia activa se crea un portafolio que tenga los mismos activos del ETF y se practican rebalanceos tomando como referencia los precios diarios y usando las siguientes condiciones:
* Disminuir en un 2.5% la posición en títulos de aquellos activos cuyo precio disminuyó un 5% o más
* Aumentar en un 2.5% la posición en títulos de aquellos activos cuyo precio aumentó un 5% o más
* Considera pagar las mismas comisiones por rebalanceo para cada cambio de posición con cada activo según aplique
* El orden de recompra será aquel activo con mayor crecimiento el día anterior



No se puede decir que una estrategia siempre va a ser mejor que la otra, ya que esto depende mucho del administrador de inverisiones que este llevandolas a cabo. Mucho influirá en el tipo de mercado que se utilice para indexar, se obtendrán resultados diferentes si solo se usan economías emergentes a economías desarrolladas. A pesar de que las economías emergentes ofrecen mayores rendimientos estos vienen de la mano del riesgo de los mismos mercados, por lo que esto podría resultar en mayores perdidas a las de economías desarrolladas. Es por esto que se buscará un equilibrio entre los rendimientos y el riesgo que se toma. 

Parte de lo complicado de las estrategias activas es la complejidad con la que se hace el rebalanceo y estos pueden ser muy subjetivos. Esto debido a que los criterios del rebalanceo recaen en el administrador de inversiones, por lo que puede cometer errores humanos o simplemente depreciar datos que tengan graves consecuencias. Por otro lado, la periodicidad con la que se haga el rebalanceo siempre va a tener resultados diferentes aunque se utilicen el resto de los criterios iguales. 

Se suele emplear el método de Sharpe para hacer calculos más precisos de las inversiones. Sin embargo, los impuestos significan gastos adicionales que disminuyen los rendimientos de las inversiones que no son tan faciles de calcular ya que cada economía tiene sus propios lineamientos para estos. Si a esto se le agrega la volatilidad que se ve en los mercados emergentes pueden resultar en el fracaso de un portafolio. Aunque las inversiones pasivas tienen el gran riesgo de quedarse desactualizadas y resultar el perdidas, esto no se eliminará con estrategias activas. 

En la actualidad, casi todos los administradores de fondos utilizan en cierta medida estrategias activas. Parte del exito y fracaso de estas estrategias en la gran diversificación que toman buscando eliminar la volatilidad. Sin embargo, cuando se diversifica tanto un portafolio no sigue los grandes movimientos de mercados en específico como los energéticos. Las estrategias pasivas son mayormente empleadas por administradores que le dan mayor importancia a los rendimientos acumulados sobre los rendimientos puros. 

El indexing viene entonces de la idea de disminuir lo más posible el riesgo, esto se ve en varias ocasiones en los índices. Es por esto que se busca armar portafolios que se asemejen a ciertos índices. Las instituciones financieras cobran mayores comisiones por llevar inversiones con este tipo de estrategias ya que no todos los administradores estan dispuestos a llevarlas a cabo. 

# 2. Objetivos 
## 2.1 General

Crear estrategias para invertir 1 millón de pesos utilizando NAFTRAC 

## 2.2 Específicos
* Generar una inversión pasiva 
* Generar una inversión activa



# 3. Datos utilizados 

Se descargaron los datos utilizando Yahoo Finance y se tomó como referencia los archivos del histórico del NAFTRAC que descargamos desde CANVAS

In [15]:
import data as dt

## 3.1 Inversión pasiva

Se busca construir una serie de tiempo en base al capital con una estrategia de inversión pasiva. Descargamos los precios de las empresas que se encuentran dentro del ETF que replica al S&P (NAFTRAC) utilizando Yahoo Finance.

In [3]:
dt.precios_mensuales.head(10)

Symbols,AMXL.MX,FEMSAUBD.MX,GFNORTEO.MX,WALMEX.MX,GMEXICOB.MX,CEMEXCPO.MX,TLEVISACPO.MX,ALFAA.MX,GAPB.MX,ASURB.MX,...,MEGACPO.MX,ALSEA.MX,CUERVO.MX,GENTERA.MX,LALAB.MX,VOLARA.MX,LABB.MX,RA.MX,ALPEKA.MX,NEMAKA.MX
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,15.813805,168.045914,106.891251,43.671349,53.690887,15.089149,75.351334,23.192703,166.901550,351.718445,...,76.367264,60.197136,31.968533,15.746818,25.930418,15.930000,20.450001,103.176376,26.630966,15.364382
2018-02-28,15.723131,161.565521,101.152481,41.494297,52.764084,12.135653,63.112366,22.261560,156.205917,324.345734,...,74.824867,63.539227,32.827698,14.374687,24.304502,16.780001,20.389999,103.381088,24.211769,13.258453
2018-03-31,15.641523,153.511307,99.337883,43.625599,49.648258,11.726258,56.577091,23.156401,154.070206,297.536224,...,75.300133,62.817410,34.838715,12.377166,21.769299,14.650000,19.700001,99.616112,25.044609,14.784846
2018-04-30,15.695926,168.663605,104.602882,49.398075,50.976578,11.404591,65.665817,23.872267,166.858658,326.579041,...,77.120537,69.559998,29.910313,13.954648,21.056276,11.690000,18.190001,109.200928,26.581390,13.963466
2018-05-31,14.054701,156.300140,94.931007,47.582600,41.040298,11.560551,65.498421,20.362518,145.759018,307.479340,...,77.783844,65.410004,31.060331,14.066659,19.267260,10.300000,16.900000,101.569290,29.496323,13.359801
2018-06-30,15.024928,164.479462,108.118439,49.768772,46.733887,12.730253,74.133194,22.967489,157.724075,312.781494,...,74.882965,68.400002,27.576122,16.730301,18.158924,10.100000,16.590000,97.465576,29.298025,13.482290
2018-07-31,14.757691,171.897263,120.022835,51.736332,48.565449,13.539296,72.962883,25.214527,151.059662,328.868347,...,82.075684,64.290001,24.926971,19.177477,17.529396,13.680000,14.870000,103.570885,31.271061,15.215867
2018-08-31,14.831616,172.047684,120.854012,50.364017,47.127899,13.188386,67.799721,24.985844,176.300232,354.982269,...,82.240402,68.059998,25.963596,18.444279,18.146013,13.960000,16.540001,107.217621,27.949625,15.810238
2018-09-30,13.898289,174.059601,124.954514,54.306290,45.386475,12.808233,65.439415,23.991579,182.860977,379.489441,...,88.106224,63.669998,29.111862,18.082438,19.119875,14.060000,16.389999,107.647186,30.259756,13.921314


## 3.2 Inversión activa

Descargamos los precios diarios desde Yahoo Finance para poder analizarla más adelante.

In [4]:
# importamos los datos diarios que descargamos de Yahoo! 
dt.precios_diarios.head(10)

Symbols,AMXL.MX,FEMSAUBD.MX,GFNORTEO.MX,WALMEX.MX,GMEXICOB.MX,CEMEXCPO.MX,TLEVISACPO.MX,ALFAA.MX,GAPB.MX,ASURB.MX,...,MEGACPO.MX,ALSEA.MX,CUERVO.MX,GENTERA.MX,LALAB.MX,VOLARA.MX,LABB.MX,RA.MX,ALPEKA.MX,NEMAKA.MX
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,15.813805,168.045929,106.891251,43.671349,53.690887,15.089149,75.351334,23.192703,166.901550,351.718445,...,76.367264,60.197136,31.968533,15.746818,25.930418,15.93,20.450001,100.471176,26.630966,15.364382
2018-02-01,15.995157,167.823746,105.729195,43.193813,53.878036,14.952686,75.282837,23.560053,166.841507,354.796570,...,76.268623,60.800293,32.006298,15.578800,25.912998,15.81,20.559999,99.994469,27.136618,15.453364
2018-02-02,15.795667,169.193863,106.971695,43.671349,52.860867,14.825968,74.891518,22.994139,164.379868,351.912659,...,75.533295,61.413338,31.996857,15.168095,25.460062,15.66,20.400000,100.601189,27.255592,15.384155
2018-02-06,15.587118,166.027740,102.940262,41.770561,52.405170,14.543290,72.866364,22.450474,160.262878,341.144165,...,75.013184,60.543209,31.741943,14.925406,25.251015,15.15,19.930000,100.523178,26.373182,15.285286
2018-02-07,15.306022,162.741257,103.860970,42.660091,50.924145,14.328844,72.279358,22.500183,162.750244,333.599457,...,74.878662,61.443001,31.977972,15.112091,25.451353,15.34,20.059999,99.985809,25.927019,14.919465
2018-02-08,15.052130,158.584549,102.591644,42.351093,49.963936,13.675761,69.138901,22.042824,158.247284,321.656067,...,73.919144,60.246574,30.637299,14.617377,24.597750,15.05,20.139999,99.422432,26.065828,14.652519
2018-02-09,15.033996,155.983124,103.682190,41.751839,50.720711,13.607529,68.630173,22.649326,154.018753,322.209534,...,73.551476,60.572872,31.071602,14.430694,24.667431,14.54,20.250000,96.492897,25.808043,14.771162
2018-02-12,15.052130,155.705383,103.288879,41.995289,51.453079,13.441820,69.344353,22.191963,157.080795,324.840973,...,73.937073,60.523434,31.553108,14.542704,24.510643,14.85,20.070000,98.373688,26.472328,14.682179
2018-02-13,15.115605,156.233078,103.092224,42.051468,52.079666,13.432074,68.894318,22.609554,156.557571,326.346008,...,74.026749,61.571545,31.203781,14.430694,25.076813,15.03,20.160000,97.558960,26.571476,13.545176


# 4. Procesos 

En esta sección se mostrarán brevemente los procesos y funciones empleadas para obtener los resultados. 

Inversión pasiva 

In [5]:
# importamos .py
import functions
import visualizations as vs
import main as main
from data import dataframes, precios_mensuales, capital, comision

Recordamos el capital a invertir

In [6]:
print(capital)

1000000


Usamos NAFTRAC al 31/01/2018 como referencia para obtener los pesos de cada activo

In [7]:
#peso%/100
main.weigths.head(10)

0    0.1331
1    0.1269
2    0.0951
3    0.0760
4    0.0754
5    0.0744
6    0.0621
7    0.0268
8    0.0251
9    0.0244
Name: Peso (%), dtype: float64

Teniendo los pesos los vamos a multiplicar por el capital para obtener lo que vamos a destinar a cada activo

In [8]:
ponderaciones=main.weigths*capital
ponderaciones[:10]

0    133100.0
1    126900.0
2     95100.0
3     76000.0
4     75400.0
5     74400.0
6     62100.0
7     26800.0
8     25100.0
9     24400.0
Name: Peso (%), dtype: float64

Usando los precios NAFTRAC al 31/01/2018 vemos cuantos titulos se obtienen de cada activo, esto lo hacemos dividiendo las ponderaciones entre el precio. 

In [9]:
precio=precios_mensuales.iloc[0,:]

titulos=[ponderaciones[i]/precio[i] for i in range(0, len(ponderaciones))]
titulos=[m.trunc(titulos[i]) for i in range(0, len(ponderaciones))]

titulos[:10]

[8416, 755, 889, 1740, 1404, 4930, 824, 1155, 150, 69]

Calculamos las comisiones a pagar al comprar los títulos anteriormente mensionados

In [10]:
comisiones=sum(titulos*precio*comision)
comisiones

1216.150759602785

Calculamos el CASH que quedó disponible después de comisiones y le sumamos el cash

In [11]:
cash=capital- sum(titulos*precio)- comisiones
cash

25863.241558169128

Calculamos el valor del portafolio inicial 

In [12]:
port_ini= sum(titulos*precio)+ cash
port_ini

998783.8492403972

Obtenemos el valor de todos los portafolios al final de cada mes

In [13]:
port_fin=[sum(precios_mensuales.iloc[i,:]*titulos) for i in range(1, len(precios_mensuales))]
port_fin.insert(0, port_ini)

port_fin[:10]

[998783.8492403972,
 917166.2670726776,
 892557.6764717102,
 938660.55991745,
 869428.4448413849,
 934387.8748626709,
 975827.4590158463,
 976485.9187431335,
 975080.1063079834,
 866324.3744764328]

Mostramos como evolucionan los rendimientos ,rendimientos acumulados y las ganancias o perdidas con el paso de los meses

In [14]:
vs.df.head(10)

,TimeStamp,Capital,Rends,Rends_acums,ganancias/perdidas
0,2018-01-31,998783.849240,0.000000,0.000000,-1216.150760
1,2018-02-28,917166.267073,-0.081717,-0.081717,-82833.732927
2,2018-03-31,892557.676472,-0.026831,-0.106356,-107442.323528
3,2018-04-30,938660.559917,0.051653,-0.060196,-61339.440083
4,2018-05-31,869428.444841,-0.073756,-0.129513,-130571.555159
5,2018-06-30,934387.874863,0.074715,-0.064474,-65612.125137
6,2018-07-31,975827.459016,0.044349,-0.022984,-24172.540984
7,2018-08-31,976485.918743,0.000675,-0.022325,-23514.081257
8,2018-09-30,975080.106308,-0.001440,-0.023733,-24919.893692
9,2018-10-31,866324.374476,-0.111535,-0.132621,-133675.625524


# 4. Conclusiones 

Podemos observar que el rendimiento, en general, es negativo por lo que podemos concluir que mensualmente hubieron perdidas en el capital. A pesar de tener rendimientos positivos, en el acumulado se traduce aún así a perdidas. Esto se puede deber a la crisis que se vio por la pandemia y que hasta la fecha seguimos viviendo sus consecuencias. Podemos decir que la estrategia planteada no es necesariamente buena porque seguimos perdiendo dinero constantemente. 

En cuanto a la parte del codigo experimentamos diversas dificultades, desde el cambio de ambientes que tenemos dentro del equipo y obtener paths que funcionaran en los diferentes ambientes es casi imposible, por lo que recurrimos a incluir lineas de codigo especiales para cada ambiente. En varios momentos nos funcionaba correctamente todos los scripts, pero al querer mostrar las cosas en este cuaderno de jupyter se generaban errores innesperados. El tiempo para correr por primera vez este cuaderno de jupyter es bastante extenso, sin embargo, si se hacen pequeñas modificaciones ya no es tan tardado correrlas. 

En general nunca habiamos trabajado con ambientes, por lo que es uno de nuestros mayores aprendizajes ya que ambos tuvimos problemas con esta parte. También cabe destacar que el trabajo en equipo no fue el mejor ya que no se contó con la colaboración de uno de los integrantes, sin embargo, consideramos que es parte de la experiencia de este tipo de trabajos. 